# Via HuggingFace's API

model zoo: https://huggingface.co/docs/api-inference/detailed_parameters

quicktour: https://huggingface.co/docs/api-inference/quicktour

In [1]:
import os
import requests
import json

In [2]:
HUGGINGFACEHUB_API_TOKEN=os.environ.get("HUGGINGFACEHUB_API_TOKEN")
headers = {"Authorization": f"Bearer {HUGGINGFACEHUB_API_TOKEN}"}

In [8]:
def query(payload, model_name):
    data = json.dumps(payload)
    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))

In [10]:
# Q&A model
model_name = "deepset/roberta-base-squad2"
query(
    {
        "inputs": {"question": "What's my name?", 
                   "context": "My name is Clara and I live in Berkeley.", }
    }, 
    model_name=model_name
)

{'error': 'overloaded'}

In [12]:
# Summarization
model_name = "facebook/bart-large-cnn"
query(
    {
        "inputs": "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, "
        "and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. "
        "During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, "
        "a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. "
        "It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, "
        "it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, "
        "the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.",
        "parameters": {"do_sample": False},
    },
    model_name=model_name
)

[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world.'}]

In [13]:
# Sentence similarity
model_name = "sentence-transformers/all-MiniLM-L6-v2"
query(
    {
        "inputs": {
            "source_sentence": "That is a happy person",
            "sentences": ["That is a happy dog", "That is a very happy person", "Today is a sunny day"],
        }
    }, 
    model_name=model_name
)

[0.6945773363113403, 0.9429150819778442, 0.2568760812282562]

# Via LangChain

In [1]:
text = "What would be a good company name for a company that makes colorful socks?"

## HuggingFaceHub

In [65]:
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate, LLMChain

In [68]:
llm_hf = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.01, "max_length":500})

In [69]:
llm_hf(text)

ValueError: Error raised by inference API: Internal Server Error

## LLMChain

In [49]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [50]:
question = "Who won the Superbowl the year Paul McCartney sang during the half-time show?"

In [51]:
llm_hf_chain = LLMChain(prompt=prompt, llm=llm_hf)

In [52]:
llm_hf_chain.run(question)

' First, the Superbowl is a football game. Second, the half-time show is a musical'